In [1]:
import sys

sys.path.insert(0, "../src")

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

nyc_app_token = os.getenv("NYC_OPENDATA_APPTOKEN")

In [3]:
from data_downloader import OpenDataDownloader, GeometryFormatter

loader = OpenDataDownloader(nyc_app_token)

In [4]:
crashes = loader.load_data("crashes", limit=10000)
streets = loader.load_data("centerline")

In [5]:
crashes_gdf = GeometryFormatter(crashes).from_lat_long()
streets_gdf = GeometryFormatter(streets).from_geometry_column("the_geom")

In [6]:
from data_helpers import CollisionRatesCalculator

collision_rates = CollisionRatesCalculator(
    crashes_df=crashes_gdf, streets_df=streets_gdf
).calculate_collision_rates()

In [17]:
collision_rates.head()

,physicalid,collision_rate
0,-1,0.0
1,10000,0.0
2,100000,0.0
3,100001,0.0
4,100003,0.0
